# An introduction to CSP-style concurrency with _aiochan_

## Intro

* Why do you want concurrent programs
    * dealing with multiple things at once (at least logically)
    * potential to decrease latency, increase throughput
    * potential to make your logic much cleaner (but also much messier if you do it wrong)
    * concurrency is not parallelism (but enables it)

* What is CSP
    * Processes: something that computes, a delineated group of code that runs
    * Sequential: statements or expressions that execute or evaluate in sequence, or at least in a deterministic order
    * Communicating: IO is done only by explicitly passing stuff around -- no locks or critical region or what not

## Basic operations

* The sad reality of concurrency in python
    * Not initially designed for concurrency
    * When you start python, you are not in a situation where you can execute concurrent instructions directly
    * Explicitly delaring concurrent environment: processes, threads, asyncio loop
    * How we can keep things as simple as possible now
    * examples: concurrent programs (but non-communicating)

* The basic IO device: channel
    * channel IS the IO device for CSP
    * get and put
    * fan-in and fan-out
    * passing channels around

* The paramount `select`
    * non-determinism or choice
    * example: stopping
    * `select` is non-trivial
        * Java was designed with threads in mind, but needs java.util.concurrent as a library
        * Clojure was designed with concurrency in mind, has all the java primitives and libraries to use, has its additional (primitive) refs (atoms, agents, event stm (software transactional memory)), but _still_ needs core.async as a library
        * You can say that the whole core.async is very much built to enable `select` (called `alt!`, `alts!`, `alt!!`, `alts!!` there)

* Understanding channel buffering
    * why buffer: even more concurrency, even more control
    * the basics: fixed buffer, dropping buffer, sliding buffer
    * channel and locks and semaphores

## Writing concurrent programs

* primitives, combinations, abstractions
    * primitives: channels
    * combinations: `put`, `get`, `select`
    * abstractions: "megachannels", something in, something out

* creating channels
    

* combining for dataflow
    

* Timing, backpressure, reactive programming

## Parallelism, non-concurrent programs

* The various forms of pipe

* How to get data in and out, from non-concurrent programs